CART (Classification and Regression Trees)

In [5]:
install.packages(c("rpart", "randomForest", "gbm", "caret", "MASS"))
library(rpart)
library(randomForest)
library(gbm)
library(caret)
library(MASS)

Installing packages into ‘/opt/homebrew/lib/R/4.5/site-library’
(as ‘lib’ is unspecified)

randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.

Loaded gbm 2.2.2

This version of gbm is no longer under development. Consider transitioning to gbm3, https://github.com/gbm-developers/gbm3

Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:randomForest’:

    margin


Loading required package: lattice



1. `formula` has the syntax `response ~ predictors`
2. `data`
3. `method`
4. `control`

In [3]:
rpart(formula, data, method, control)

ERROR: Error in rpart(formula, data, method, control): could not find function "rpart"
